# Plot the MCMC results

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
PATH_h5file_chains = './mcmc/chain.h5'
PARAMETER_NAMES = [
    r'$B_G\;[\mathrm{GeV}^{-2}]$',
    r'$\alpha_{\rm shadowing}$',
    r'$y_{{\rm loss},2}$',
    r'$y_{{\rm loss},4}$',
    r'$y_{{\rm loss},6}$',
    r'$\sigma_{y_{\rm loss}}$',
    r'$\alpha_{\rm rem}$',
    r'$\lambda_B$',
    r'$\sigma_x^{\rm string}\;[{\rm fm}]$',
    r'$\sigma_\eta^{\rm string}$',
    r'$\alpha_{{\rm string}\;{\rm tilt}}$',
    r'$\alpha_{\rm preFlow}$',
    r'$\eta_0$',
    r'$\eta_2$',
    r'$\eta_4$',
    r'$\zeta_{\rm max}$',
    r'$T_{\zeta,0}\;[{\rm GeV}]$',
    r'$\sigma_{\zeta,+}\;[{\rm GeV}]$',
    r'$\sigma_{\zeta,-}\;[{\rm GeV}]$',
    r'$e_{\rm sw}\;[{\rm GeV}/{\rm fm}^3]$'
    ]

# Open the HDF5 file in read mode
with h5py.File(PATH_h5file_chains, 'r') as hf:
    # Iterate over the keys in the file
    for key in hf.keys():
        # Access the dataset corresponding to the key
        dataset = hf[key]
        
        n_walkers = dataset.shape[0]
        n_steps = dataset.shape[1] # this is the originally set value divided by 10
        n_parameters = dataset.shape[2]

        # Create a 3D array to store the data
        data_array = np.zeros((n_walkers, n_steps, n_parameters))

        # Iterate through the dataset and store the data in the array
        for i in range(n_walkers):
            for j in range(n_steps):
                data_array[i, j, :] = dataset[i, j, :]
        
        # Remove parameters from data_array [wrong zeta/s(mu_B) parameters]
        data_array = np.delete(data_array, [16,17], axis=2)
        # reset the number of parameters after deleting some of them
        n_parameters = data_array.shape[2]


In [ ]:
num_columns = 3

# Calculate the number of rows needed based on the number of parameters and columns
num_rows = (n_parameters + num_columns - 1) // num_columns

fig, axs = plt.subplots(num_rows, num_columns, figsize=(10, 7), sharex=False)
axs = axs.flatten()

for param_index in range(n_parameters):
    # Plot each walker's data for the parameter
    for walker_index in range(n_walkers):
        axs[param_index].plot(range(n_steps), data_array[walker_index, :, param_index], color='g', alpha=0.5)

    axs[param_index].set_ylabel(PARAMETER_NAMES[param_index])
    axs[param_index].set_xlim([0.,n_steps])

# Specify indices for which to display x-axis labels
indices_with_labels = [17, 18, 19]

# Set x-axis labels and numbers for specified indices
for i, ax in enumerate(axs):
    if i in indices_with_labels:
        ax.set_xlabel('Steps')
        ticks = axs[param_index].get_xticks()
        axs[param_index].set_xticks(ticks)
        axs[param_index].set_xticklabels([int(tick) for tick in ticks])
    else:
        ax.set_xticklabels([])

# Hide the empty subplots
for i in range(n_parameters, num_columns * num_rows):
    fig.delaxes(axs[i])

plt.tight_layout()
plt.savefig("chain.pdf")

In [ ]:
samples = data_array[:,:,:].reshape((-1,n_parameters))

# Create subplots
fig, axes = plt.subplots(n_parameters, n_parameters, figsize=(25, 25), sharex='col', sharey='row')

# Loop over each parameter pair
for i in range(n_parameters):
    for j in range(n_parameters):
        ax = axes[i, j]

        if i == j:
            ax = fig.add_subplot(n_parameters, n_parameters, i * n_parameters + j + 1)
            ax.hist(samples[:, i], bins=20, color='g', histtype='step', density=True)
            ax.set_yticks([])
            ax.set_yticklabels([])
            if i != n_parameters-1:
                ax.set_xticks([])
                ax.set_xticklabels([])
            if i == n_parameters-1:
                ax.tick_params(axis='x', rotation=45, labelsize=10)

            # Calculate percentiles
            percentiles = np.percentile(samples[:, i], [16, 50, 84])
            median = percentiles[1]
            upper = percentiles[2] - median
            lower = median - percentiles[0]

            ax.annotate(fr'${median:.2f}_{{-{lower:.2f}}}^{{+{upper:.2f}}}$', xy=(0.5, 1.05), xycoords='axes fraction', ha='center')

        # Only fill the lower triangle
        if i > j:
            ax.hist2d(samples[:, j], samples[:, i], bins=20, cmap='Greens')

        # Set labels on the lowest and leftmost plots
        if i == n_parameters - 1:
            ax.set_xlabel(PARAMETER_NAMES[j])
        if j == 0:
            ax.set_ylabel(PARAMETER_NAMES[i])

        if i < j:
            ax.axis('off')

# Remove space between subplots
plt.subplots_adjust(wspace=0, hspace=0)

# Remove ticks and labels for the first and last plots
axes[0, 0].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
axes[n_parameters - 1, n_parameters - 1].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

# Rotate x-axis tick labels
for ax in axes[-1]:
    ax.tick_params(axis='x', rotation=45, labelsize=10)

# Rotate y-axis tick labels
for ax in axes[:, 0]:
    ax.tick_params(axis='y', rotation=45, labelsize=10)

plt.tight_layout()
plt.savefig("posterior.pdf")